In [10]:
from configobj import ConfigObj
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.interpolate import RegularGridInterpolator
import pyccl as ccl

#from HiCOLA import Frontend

In [11]:
def wr_funct(T_CMB):
    return 4.48150052e-7*T_CMB**4 *(1+ 3.044*7/8 * (4/11)**(4/3))

T_CMB = 2.72548 # K
print(wr_funct(T_CMB))

4.182345109176885e-05


# Get Latin Hypercube

Latin hypercube already saved in:
'/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/cosmo_newdesign_tracker'


In [12]:
sample_new = np.loadtxt('/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/cosmo_validation').T


# Get Power Spectra

In [11]:
# Create cola folders - only need to do this once

#for i in range(len(sample_new[0])):
    # CAREFUL!! might not need this next step
    #command = 'cd; rm -r /home/c2042999/HiCOLA_env/Cubic_galileon_emu/Output_backend/COLA_{}_GR_tracker'.format(str(i))
    #os.system(command)
    #os.system("cd; cd /home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Output_backend/; mkdir COLA_{}_GR_tracker".format(str(i)))


# Run Frontend

In [4]:
def rewriteluafile(k,txt,boxsize,n_particles,n_mesh):
    """
    Takes in txt file with (omega_m,n_s,A_s,E_ds,f_phi) for all k
    For a certain backend file number (a certain k) it creates a lua file that makes HiCOLA output in a separate folder
    """

    wb = 0.0223
    wr = wr_funct(T_CMB)
    
    hcube = np.loadtxt(txt).T
    n_s = hcube[1][k]
    A_s = hcube[2][k]
    h = hcube[3][k]
    wm = hcube[0][k]*h**2

    f_phi = hcube[4][k]
    
    # Derived params
    Omg_r = wr/h**2
    Omg_m = wm/h**2
    Omg_b = wb/h**2
    Omg_Lambda = (1-f_phi)*(1-Omg_m-Omg_r)
    Omg_DE = (1-Omg_m-Omg_r)
    
    a= open('/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Hi-COLA_params_emu.lua','r')
    b = a.readlines()
    c = open('/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Backend_input_parameter_files/luafiles_validation/Hi-COLA_params_emu_{}_GR_tracker.lua'.format(str(k)),'w')
    for i in range(len(b)):
        if i==19:
            c.write('simulation_name = "my_new_Hi-COLA_simulation_{}_GR_tracker"'.format(str(k))+'\n')
        elif i==21:
            c.write('simulation_boxsize = '+str(boxsize)+'\n')
        elif i==38:
            c.write('cosmology_model = "LCDM"'+'\n')
        elif i== 40:
            c.write('cosmology_OmegaCDM = '+str((wm-wb)/h**2)+'\n')
        elif i== 42:
            c.write('cosmology_Omegab = '+str(wb/h**2)+'\n')
        elif i== 46:
            c.write('cosmology_OmegaLambda = '+str(Omg_DE)+'\n')
        elif i== 50:
            c.write('cosmology_TCMB_kelvin = '+str(T_CMB)+'\n')
        elif i== 52:
            c.write('cosmology_h = '+str(h)+'\n')
        elif i== 54:
            c.write('cosmology_As = '+str(A_s)+'\n')
        elif i== 56:
            c.write('cosmology_ns = '+str(n_s)+'\n')
        elif i== 95:
            c.write('gravity_model = "GR"'+'\n')
        elif i== 208:
            c.write('particle_Npart_1D = '+str(n_particles)+'\n')
        elif i== 210:
            c.write('particle_allocation_factor = '+str(2.0)+'\n')
        elif i== 222:
            c.write('output_folder = "/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Output_backend_validation/COLA_{}_GR_tracker"'.format(str(k))+'\n')
        elif i== 229:
            c.write('timestep_nsteps = {100}'+'\n')
        elif i== 247:
            c.write('ic_random_seed ='+str(108)+'\n')
        elif i== 251:
            c.write('ic_fix_amplitude = true'+'\n')
        elif i== 253:
            c.write('ic_reverse_phases = false'+'\n')
        elif i== 269:
            c.write('ic_input_filename = "/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Backend_input_parameter_files/P_k_z_validation/Pk_lin_z49_LCDM_backscaled_val.dat"'+'\n')
        elif i== 271:
            c.write('ic_input_redshift = 49.0'+'\n')
        elif i== 312:
            c.write('force_nmesh = '+str(n_mesh)+'\n')
        elif i== 345:
            c.write('pofk_nmesh = '+str(n_mesh)+'\n')
        #elif i== 359:
        #    c.write('pofk_multipole_nmesh = '+str(n_mesh)+'\n')
        #elif i== 375:
        #    c.write('bispectrum_nmesh = '+str(n_mesh)+'\n')
        else:
            c.write(b[i])

    c.close()
    a.close()



In [8]:
def rewriteluafile_phasepair(k,txt,boxsize,n_particles,n_mesh):
    """
    Takes in txt file with (omega_m,n_s,A_s,E_ds,f_phi) for all k
    For a certain backend file number (a certain k) it creates a lua file that makes HiCOLA output in a separate folder
    """

    wb = 0.0223
    wr = wr_funct(T_CMB)
    
    hcube = np.loadtxt(txt).T
    n_s = hcube[1][k]
    A_s = hcube[2][k]
    h = hcube[3][k]
    wm = hcube[0][k]*h**2
    
    f_phi = hcube[4][k]
    
    # Derived params
    Omg_r = wr/h**2
    Omg_m = wm/h**2
    Omg_b = wb/h**2
    Omg_Lambda = (1-f_phi)*(1-Omg_m-Omg_r)
    Omg_DE = (1-Omg_m-Omg_r)
    
    a= open('/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Hi-COLA_params_emu.lua','r')
    b = a.readlines()
    c = open('/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Backend_input_parameter_files/luafiles_validation/Hi-COLA_params_emu_{}_GR_tracker_phasepair.lua'.format(str(k)),'w')
    for i in range(len(b)):
        if i==19:
            c.write('simulation_name = "my_new_Hi-COLA_simulation_{}_GR_tracker_phasepair"'.format(str(k))+'\n')
        elif i==21:
            c.write('simulation_boxsize = '+str(boxsize)+'\n')
        elif i==38:
            c.write('cosmology_model = "LCDM"'+'\n')
        elif i== 40:
            c.write('cosmology_OmegaCDM = '+str((wm-wb)/h**2)+'\n')
        elif i== 42:
            c.write('cosmology_Omegab = '+str(wb/h**2)+'\n')
        elif i== 46:
            c.write('cosmology_OmegaLambda = '+str(Omg_DE)+'\n')
        elif i== 50:
            c.write('cosmology_TCMB_kelvin = '+str(T_CMB)+'\n')
        elif i== 52:
            c.write('cosmology_h = '+str(h)+'\n')
        elif i== 54:
            c.write('cosmology_As = '+str(A_s)+'\n')
        elif i== 56:
            c.write('cosmology_ns = '+str(n_s)+'\n')
        elif i== 95:
            c.write('gravity_model = "GR"'+'\n')
        elif i== 208:
            c.write('particle_Npart_1D = '+str(n_particles)+'\n')
        elif i== 210:
            c.write('particle_allocation_factor = '+str(2.0)+'\n')
        elif i== 222:
            c.write('output_folder = "/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Output_backend_validation/COLA_{}_GR_tracker"'.format(str(k))+'\n')
        elif i== 229:
            c.write('timestep_nsteps = {100}'+'\n')
        elif i== 247:
            c.write('ic_random_seed ='+str(108)+'\n')
        elif i== 251:
            c.write('ic_fix_amplitude = true'+'\n')
        elif i== 253:
            c.write('ic_reverse_phases = true'+'\n')
        elif i== 269:
            c.write('ic_input_filename = "/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Backend_input_parameter_files/P_k_z_validation/Pk_lin_z49_LCDM_backscaled_val.dat"'+'\n')
        elif i== 271:
            c.write('ic_input_redshift = 49.0'+'\n')
        elif i== 312:
            c.write('force_nmesh = '+str(n_mesh)+'\n')
        elif i== 345:
            c.write('pofk_nmesh = '+str(n_mesh)+'\n')
        #elif i== 359:
        #    c.write('pofk_multipole_nmesh = '+str(n_mesh)+'\n')
        #elif i== 375:
        #    c.write('bispectrum_nmesh = '+str(n_mesh)+'\n')
        else:
            c.write(b[i])

    c.close()
    a.close()

# Same as above
txt = '/home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/cosmo_validation'



In [9]:
"""Re-write lua file"""
for i in range(len(sample_new[0])):
    rewriteluafile(i,txt,400,512,1536)
    # phase pair
    rewriteluafile_phasepair(i,txt,400,512,1536)


In [ ]:
"""Create Pk"""
for i in range(len(sample_new[0])):
    os.system("cd; cd /home/c2042999/HiCOLA_env/Hi-COLA/HiCOLA/FML/COLASolver; mpirun -np 32 nbody /home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Backend_input_parameter_files/luafiles_validation/Hi-COLA_params_emu_{}_GR_tracker.lua".format(str(i)))
    


#=====================================================
#           ________________  .____         
#           \_   _____/     \ |    |      
#            |    __)/  \ /  \|    |      
#            |     \/    Y    \    |___   
#            \___  /\____|__  /_______ \ 
#                \/         \/        \/ 
#
# Initializing FML, MPI and FFTW
# MPI is enabled. Running with 32 MPI tasks
# OpenMP is enabled. Main task has 80 threads availiable
# MPI + Threads is working
# FFTW is enabled. Thread support is enabled
#
# List of tasks:
# Task    0 [pulsar-cn-7]
#     x-domain [       0 ,  0.03125)
# Task    1 [pulsar-cn-7]
#     x-domain [ 0.03125 ,   0.0625)
# Task    2 [pulsar-cn-7]
#     x-domain [  0.0625 ,  0.09375)
# Task    3 [pulsar-cn-7]
#     x-domain [ 0.09375 ,    0.125)
# Task    4 [pulsar-cn-7]
#     x-domain [   0.125 ,  0.15625)
# Task    5 [pulsar-cn-7]
#     x-domain [ 0.15625 ,   0.1875)
# Task    6 [pulsar-cn-7]
#     x-domain [  0.1875 ,  0.21875)
# Task    7 [pulsa

In [ ]:
"""Create Pk"""
for i in range(len(sample_new[0])):
    os.system("cd; cd /home/c2042999/HiCOLA_env/Hi-COLA/HiCOLA/FML/COLASolver; mpirun -np 32 nbody /home/c2042999/HiCOLA_env/Cubic_galileon_emu-ICGweek/Backend_input_parameter_files/luafiles_validation/Hi-COLA_params_emu_{}_GR_tracker_phasepair.lua".format(str(i)))
    